# API Discovery with Knowledge Graphs

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Setup

In [2]:
from rdflib import Dataset
import pandas as pd

from faiss import IndexFlatL2, IndexFlat
import numpy as np
import tqdm
from langchain_openai import OpenAIEmbeddings
from helper import parameterize_sparql

In [3]:
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_colwidth", None)

In [4]:
from helper import get_openai_api_key
openai_api_key = get_openai_api_key()

In [5]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

In [6]:
graph = Dataset(default_union=True)
graph.parse("./ro_shared_data/odata_knowledge_graph.ttl",
            format="turtle")

<Graph identifier=urn:x-rdflib:default (<class 'rdflib.graph.Graph'>)>

## Generating the embedding strings

In [7]:
q_api_properties = """PREFIX odata: <http://example.org/odata#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT
    ?entity_set ?property_label

WHERE {
    BIND(<http://data.example.org/Service/API_PURCHASEORDER_2/EntitySet/PURCHASEORDER> 
    as ?entity_set_uri)
    ?entity_set_uri rdf:type odata:EntitySet.
    ?entity_set_uri odata:name ?entity_set.
    ?entity_set_uri odata:entityType ?entity_type_uri.


    # Properties on the EntitySet
    {
        ?entity_type_uri odata:property ?property_uri.
        ?property_uri odata:label ?property_label.
        # ?property_uri odata:selectProperty true
    }
}
"""

In [8]:
df_dict = {
    "entity_set": [],
    "property_label": [],
}
for row in graph.query(q_api_properties):
    df_dict["entity_set"].append(str(row.entity_set))
    df_dict["property_label"].append(str(row.property_label))


In [9]:
pd.DataFrame(df_dict).head(15)

,entity_set,property_label
0,PurchaseOrder,Company Code
1,PurchaseOrder,Fixed Exchange Rate
2,PurchaseOrder,Customer
3,PurchaseOrder,Goods Supplier
4,PurchaseOrder,Incoterms Version
5,PurchaseOrder,Release Strategy
6,PurchaseOrder,Your Reference
7,PurchaseOrder,CD Percentage 1
8,PurchaseOrder,Down Payment Amount
9,PurchaseOrder,Days 1


In [10]:
q_embedding_string = """
PREFIX odata: <http://example.org/odata#>
SELECT 
?entity_set_uri (CONCAT(CONCAT(CONCAT("entity set: ", ?entity_set_name), "; properties: "), group_concat(?property_label;separator=", ")) 
    AS ?embedding_string)
WHERE
{
	?entity_set_uri a odata:EntitySet ;
        odata:name ?entity_set_name ;
        odata:entityType/odata:property/odata:label ?property_label .
}
GROUP BY ?entity_set_uri ?entity_set_name
"""

In [11]:
str(next(iter(graph.query(q_embedding_string))).embedding_string)

'entity set: BillingDocument; properties: Abs.Exch.Rate Acct., Document Number, Set Exchange Rate, Posting Status, Posting Status, Indirect Quotation, Addit. Value Days, Assignment, Combination Criteria, Billing Document, Billing Category, Billing Date, Canceled, Billing Type, Issue Type, Canceled Bill. Doc., City Code, Company Code, Contract Account, Dest. Country/Region, County Code, Created By, Created On, Time, Credit Control Area, Acct Assmt Grp Cust., Customer Group, Terms of Payment, Customer Price Group, Tax Class.1 Customer, Tax Class.2 Customer, Tax Class.3 Customer, Tax Class.4 Customer, Tax Class.5 Customer, Tax Class.6 Customer, Tax Class.7 Customer, Tax Class.8 Customer, Tax Class.9 Customer, Distribution Channel, Division, Reference, Dunning Area, Dunning Block, Dunning Key, Translation Date, Exchange Rate Type, Fiscal Year, Fixed Value Date, Incoterms, Incoterms Location 1, Incoterms Location 2, Incoterms (Part 2), Incoterms Version, Inv. List Bill. Date, Invoice List S

## Generate Embeddings

In [12]:
embeddings = []
entity_set_uris = []

for row in tqdm.tqdm(graph.query(q_embedding_string)):
    embedding = embedding_model.embed_query(row.embedding_string)
    embeddings.append(embedding)
    entity_set_uris.append(str(row.entity_set_uri))

100%|██████████| 100/100 [00:02<00:00, 36.16it/s]


In [13]:
xb = np.array(embeddings).astype("float32")
index = IndexFlatL2(xb.shape[1])
index.add(xb)

In [14]:
import pickle

with open("../ro_shared_data/entity_sets_index.pickle", "wb") as file:
    pickle.dump(index, file)

with open("../ro_shared_data/entity_set_uris.pickle", "wb") as file:
    pickle.dump(entity_set_uris, file)

## Query the index

In [15]:
def query_index(
    index: IndexFlat,
    entity_set_uris: list[str],
    embedding_model: OpenAIEmbeddings,
    query: str,
    top: int = 5,
) -> list[str]:
    x_query = np.array([embedding_model.embed_query(query)])
    _, indices = index.search(x_query, top)
    return [entity_set_uris[i] for i in indices[0]]

In [16]:
query_index(
    index=index,
    entity_set_uris=entity_set_uris,
    embedding_model=embedding_model,
    query="""Create a purchase order for 5 pencils in 
    purchasing group 002 and purchasing organization 3000""",
    top=5,
)

['http://data.example.org/Service/API_PURCHASEORDER_2/EntitySet/PURCHASEORDER',
 'http://data.example.org/Service/API_PURCHASEORDER_2/EntitySet/PURCHASEORDERITEM',
 'http://data.example.org/Service/API_PURCHASEREQUISITION_2/EntitySet/PURCHASEREQNITEM',
 'http://data.example.org/Service/API_CNTRLREQFORQUOTATION/EntitySet/CNTRLREQFORQUOTATIONBIDDER',
 'http://data.example.org/Service/API_PLANNEDORDER/EntitySet/PLANNEDORDERHEADER']

## Enhace retrieval with process information

In [17]:
q_get_process_dependencies = """
    PREFIX pr: <http://example.org/process#>
    PREFIX odata: <http://example.org/odata#>
    SELECT DISTINCT ?entitySetA ?entitySetB ?nameA ?nameB
    WHERE {
        {
        VALUES ?entitySetA { var:::entity_set_uris }
        ?activityA  pr:entitySet ?entitySetA ;
                    pr:hasNext ?activityB . 
        
        ?activityB pr:entitySet ?entitySetB .
        ?entitySetA odata:name ?nameA .
        ?entitySetB odata:name ?nameB .
        } 
        UNION {
        VALUES ?entitySetB { var:::entity_set_uris }
        ?activityA  pr:entitySet ?entitySetA ;
                    pr:hasNext ?activityB . 
        
        ?activityB pr:entitySet ?entitySetB .
        ?entitySetA odata:name ?nameA .
        ?entitySetB odata:name ?nameB .
        }
    }
    """

In [18]:
def get_process_dependencies(
    entity_set_uris: list[str], graph: Dataset
) -> list[tuple[str, str, str, str]]:
    entity_set_uris = " ".join([f"<{uri}>" for uri in entity_set_uris])
    return [
        (str(row.entitySetA), str(row.entitySetB), str(row.nameA), str(row.nameB))
        for row in graph.query(
            parameterize_sparql(
                query=q_get_process_dependencies,
                parameters={"entity_set_uris": entity_set_uris},
            )
        )
    ]

In [19]:
retrieved_entity_set_uris = query_index(
    index=index,
    entity_set_uris=entity_set_uris,
    embedding_model=embedding_model,
    query="""Create a purchase order for 5 pencils in
    purchasing group 002 and purchasing organization 3000""",
    top=5,
)

get_process_dependencies(entity_set_uris=retrieved_entity_set_uris, 
                         graph=graph)

[('http://data.example.org/Service/API_PURCHASEREQUISITION_2/EntitySet/PURCHASEREQN',
  'http://data.example.org/Service/API_PURCHASEORDER_2/EntitySet/PURCHASEORDER',
  'PurchaseReqn',
  'PurchaseOrder')]

## Putting everything together

In [20]:
def discover_apis_and_process(
    query: str,
    graph: Dataset,
    index: IndexFlat,
    entity_set_uris: list[str],
    embedding_model: OpenAIEmbeddings,
) -> dict:
    
    retrieved_entity_set_uris = query_index(
        index=index,
        entity_set_uris=entity_set_uris,
        embedding_model=embedding_model,
        query=query,
        top=5,
    )

    dependencies = get_process_dependencies(
        entity_set_uris=retrieved_entity_set_uris, graph=graph
    )
    
    merged_entity_sets = set(retrieved_entity_set_uris)

    process_information = []
    for dependency in dependencies:
        merged_entity_sets.add(dependency[0])
        merged_entity_sets.add(dependency[1])
        process_information.append(
            f"{dependency[3]} depends on {dependency[2]}")

    return {
        "entity_sets": merged_entity_sets,
        "process_information": process_information,
    }

In [21]:
discover_apis_and_process(
    query="""Create a purchase order for 5 pencils in
    purchasing group 002 and purchasing organization 3000""",
    graph=graph,
    index=index,
    entity_set_uris=entity_set_uris,
    embedding_model=embedding_model,
)

{'entity_sets': {'http://data.example.org/Service/API_CNTRLREQFORQUOTATION/EntitySet/CNTRLREQFORQUOTATIONBIDDER',
  'http://data.example.org/Service/API_PLANNEDORDER/EntitySet/PLANNEDORDERHEADER',
  'http://data.example.org/Service/API_PURCHASEORDER_2/EntitySet/PURCHASEORDER',
  'http://data.example.org/Service/API_PURCHASEORDER_2/EntitySet/PURCHASEORDERITEM',
  'http://data.example.org/Service/API_PURCHASEREQUISITION_2/EntitySet/PURCHASEREQN',
  'http://data.example.org/Service/API_PURCHASEREQUISITION_2/EntitySet/PURCHASEREQNITEM'},
 'process_information': ['PurchaseOrder depends on PurchaseReqn']}